# Ananya Agrawal (ananyaa2)

## Assignment 3 - Meeting Scheduling

In this assignment, students will complete three consequecative tasks: (1) classify e-mail messages as to whether they are meeting-related using finite set of meeting labels; (2) extract meeting dates from meeting-related e-mails; and (3) prepare a function call using OpenAI's function calling prompt.

In [1]:
import json

data = json.load(open('training_data.json', 'r'))
print('Read %i messages' % len(data))

Read 100 messages


In [ ]:
from openai import OpenAI

client = OpenAI(api_key=api_key)


def prompt_model(prompt, response_format=None):
    completion = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[
            {'role': 'assistant', 'content': 'You are a helpful assistant.'},
            {
                'role': 'user',
                'content': prompt
            }
        ],
        response_format=response_format
    )
    return completion.choices[0].message.content.strip()

## Classify E-mails by Meeting Type

Create prompt to classify messages and score prompt performance using accuracy.

In [3]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report

# Extract relevant content from emails
def preprocess_email(email_entry):
    """Extract the email body from the full email entry."""
    parts = email_entry.split("|", 3)
    return parts[-1].strip() if len(parts) == 4 else email_entry  # Return body

emails = [preprocess_email(item["email"]) for item in data]
true_labels = [item["label"].lower() for item in data]  # Ensure uniform lowercase labels


def classify_email(email_text):
    prompt = f"""
    You are an AI trained to classify emails into one of these categories:

    (A) **Meeting invitation** - **An email that explicitly invites the recipient** to a meeting.
        - Example: "Join us for a meeting on Friday at 2 PM in Room 305."
        - **Phrases like "You're invited," "Please join us," or "We have scheduled a meeting" indicate an invitation.**
        - **If an email confirms or reminds about an already scheduled meeting, it is NOT an invitation.**

    (B) **Meeting reminder** - **A follow-up email reminding about a previously scheduled meeting.**
        - Example: "Reminder: Our weekly sync is tomorrow at 10 AM in Conference Room A."
        - **A reminder does NOT invite anyone new. If it only confirms an existing meeting, it is a REMINDER.**
        - **Phrases like "Reminder," "Don't forget," or "See you at the meeting" suggest a reminder.**

    (C) **Meeting update** - **An email modifying an existing meeting’s date, time, or location.**
        - Example: "The project review meeting has been moved from 3 PM to 4 PM on Monday."
        - **If an email modifies a meeting, classify it as 'Meeting Update' NOT an invitation.**
        - **Look for phrases like "has been rescheduled," "time has changed," or "moved to a new location."**

    (D) **Meeting cancellation** - **An email canceling a previously scheduled meeting.**
        - Example: "Tomorrow’s strategy meeting has been canceled. We will reschedule soon."

    (E) **None of the above** - **The email does NOT mention a meeting OR lacks explicit scheduling details.**
        - Example: "Attached is the latest project report for your review."
        - **If the email does NOT mention a meeting, it is 'None'. However, DO NOT classify an invitation as 'None'.**
    
    ---
    Example email:  
    "Hey team, just a reminder that we have our weekly sync on Monday at 10 AM. See you all there!"  
    **Correct Answer:** B (Meeting Reminder)

    ---
    Now classify this email:  
    "{email_text}"  
    ---
    
    **Respond with only A, B, C, D, or E.**
    """

    response = prompt_model(prompt).strip().upper()  # Normalize response to uppercase
    if response not in ["A", "B", "C", "D", "E"]:
        print(f"Unexpected response: '{response}', defaulting to 'E'")
        return "E"

    return response



# Map model response to label
def map_choice_to_label(choice):
    mapping = {
        "A": "meeting invitation",
        "B": "meeting reminder",
        "C": "meeting update",
        "D": "meeting cancellation",
        "E": "none"
    }
    return mapping.get(choice, "none")

# Run classification
predicted_labels = [map_choice_to_label(classify_email(email)) for email in emails]

# Evaluate model performance
accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Classification Accuracy: {accuracy:.2%}")

# Generate a classification report
report = classification_report(true_labels, predicted_labels, zero_division=0)
print(report)


Classification Accuracy: 94.00%
                      precision    recall  f1-score   support

meeting cancellation       1.00      1.00      1.00         1
  meeting invitation       0.56      0.83      0.67         6
    meeting reminder       0.00      0.00      0.00         2
      meeting update       1.00      0.67      0.80         3
                none       0.98      0.98      0.98        88

            accuracy                           0.94       100
           macro avg       0.71      0.70      0.69       100
        weighted avg       0.93      0.94      0.93       100



## Extract Meeting Date and Time Information

Create prompt to extract meeting date and times and score prompt performance using precision and recall.

In [4]:
import re

# Step 2: Extract Meeting Date and Time
def extract_meeting_datetime(email_text):
    prompt = f"""
    Extract all **explicit** meeting dates and times from the following email.
    
    **Format Instructions:**
    - Use **MM-DD-YYYY** for dates. If the year is missing, use **0000**.
    - Use **24-hour time format** for times. If missing, use **00:00-00:00**.
    - **If multiple dates/times exist, return them as a semicolon-separated list.**
    
    **Rules:**
    - If a date is unclear but mentions a specific day (e.g., "next Monday"), return **"none"**.
    - **Ensure extracted values strictly follow the correct format.**
    
    **Example Outputs:**
    - "Let's meet on September 19 at 3 PM." → **09-19-0000 15:00-00:00**
    - "Meeting on 10/21 at 10:30 AM - 12:00 PM" → **10-21-0000 10:30-12:00**
    - "Meet next week." → **none**
    
    Email: "{email_text}"
    
    **Respond ONLY with extracted date-time values or "none".**
    """

    response = prompt_model(prompt).strip()
    
    
    # Normalize response to ensure valid formatting
    return normalize_extracted_dates(response)


# Normalize extracted dates to match expected format
def normalize_extracted_dates(date_string):
    """Ensure extracted date-time values match MM-DD-YYYY HH:MM-HH:MM format"""
    valid_format = re.compile(r"\d{2}-\d{2}-\d{4} \d{2}:\d{2}-\d{2}:\d{2}")
    
    # Ensure response is a string and split by semicolon if multiple dates exist
    if isinstance(date_string, str):
        extracted_dates = [dt.strip() for dt in date_string.split(";") if valid_format.match(dt.strip())]
        return ";".join(extracted_dates) if extracted_dates else "none"
    
    return "none"


# Extract emails, labels, and true dates from the dataset
emails = [preprocess_email(item["email"]) for item in data]
true_labels = [item["label"] for item in data]
true_dates = [item["dates"] for item in data]  # Ensure true_dates is properly defined


# Filter dataset to exclude emails where label is 'none' or dates are empty
filtered_emails, filtered_labels, filtered_true_dates = [], [], []

for email, label, dates in zip(emails, predicted_labels, true_dates):
    # Only include if the email has a valid meeting classification and at least one date
    if label.lower() != "none" and dates.strip():
        filtered_emails.append(email)
        filtered_labels.append(label)
        filtered_true_dates.append(dates)


# Extract dates from meeting-related emails
predicted_dates = [extract_meeting_datetime(email) if "meeting" in label.lower() else "" for email, label in zip(filtered_emails, filtered_labels)]


# Flatten lists for evaluation
def flatten_date_list(date_list):
    """Ensure extracted date-time values are formatted properly."""
    return set(dates.strip() for dates in date_list if dates and isinstance(dates, str))


true_dates_flat = [flatten_date_list(dates) for dates in filtered_true_dates]
predicted_dates_flat = [flatten_date_list(dates) for dates in predicted_dates]


# Compute precision and recall
def calculate_precision_recall_f1(true_list, pred_list):
    true_positive = sum(len(t & p) for t, p in zip(true_list, pred_list))
    predicted_count = sum(len(p) for p in pred_list)
    actual_count = sum(len(t) for t in true_list)
    
    precision = true_positive / predicted_count if predicted_count else 0
    recall = true_positive / actual_count if actual_count else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) else 0

    return precision, recall, f1_score

precision, recall, f1_score = calculate_precision_recall_f1(true_dates_flat, predicted_dates_flat)

print(f"Date-Time Extraction Precision: {precision:.2%}")
print(f"Date-Time Extraction Recall: {recall:.2%}")
print(f"Date-Time Extraction F1 Score: {f1_score:.2%}")


Date-Time Extraction Precision: 91.30%
Date-Time Extraction Recall: 93.33%
Date-Time Extraction F1 Score: 92.31%


## Prepare Function Call to Check Calendar Availability

Given a date and time, prepare a function call using OpenAI's function calling framework: 
https://platform.openai.com/docs/guides/function-calling

In [5]:
import json
from datetime import datetime

def prepare_function_call(date_time, index):
    try:
        date_time = date_time.strip()
        print(f"Parsing date_time: '{date_time}'")  

        if " " not in date_time:
            print(f"Skipping due to missing space in date_time: '{date_time}'")
            return None  

        date_part, time_range = date_time.split(" ")
        month, day, year = date_part.split("-")

        if year == "0000":
            print(f"Fixing missing year for date '{date_time}'")
            year = "2024"  

        try:
            formatted_date = datetime.strptime(f"{month}-{day}-{year}", "%m-%d-%Y").strftime("%B %d, %Y")
        except ValueError:
            print(f"Skipping due to invalid date: '{date_part}'")
            return None  

        if "-" not in time_range:
            print(f"Skipping due to invalid time range: '{time_range}'")
            return None

        start_time, end_time = time_range.split("-")

        if ":" not in start_time:
            print(f"Skipping due to missing ':' in start_time: '{start_time}'")
            return None

        hour, minute = start_time.split(":")

        if end_time == "00:00":
            print(f"Assigning default time for missing time in '{date_time}'")
            start_time, end_time = "09:00", "12:00"  # Default morning time


    except ValueError as e:
        print(f"Skipping due to invalid format: {date_time} - Error: {e}")
        return None  

    timezone = "EST"  # Default to Eastern Standard Time

    function_call = {
        "name": "check_availability",
        "parameters": {
            "month": int(month),
            "day": int(day),
            "year": int(year),
            "hour": int(hour),
            "minute": int(minute),
            "timezone": timezone
        }
    }

    # Generate a structured function call response
    prompt = f"""
    Given the following structured request, return ONLY 'yes' or 'no':
    - Date: {formatted_date}
    - Time: {hour}:{minute} {timezone}
    - Function: check_availability
    Does the user have availability? Answer ONLY 'yes' or 'no'.
    """

    response = prompt_model(prompt).strip().lower()
    if response not in ["yes", "no"]:
        print(f"Unexpected response: {response}")
        return None  # Skip invalid responses

    return {"index": index, "available": response}


# Generate function calls for valid meeting invitations only
function_calls = []
for idx, (label, dates) in enumerate(zip(filtered_labels, predicted_dates)):
    if label.lower() == "meeting invitation" and isinstance(dates, str) and dates.strip():
        for date_time in dates.split(";"):
            call = prepare_function_call(date_time, idx)
            if call:  # Avoid None values
                function_calls.append(call)
                print(f"Generated Function Call: {call}")


from collections import Counter, defaultdict

grouped_calls = defaultdict(list)
for call in function_calls:
    grouped_calls[call["index"]].append(call["available"])

# Optimize the structure to count repeated responses
final_function_calls = []
for idx, responses in grouped_calls.items():
    response_counts = Counter(responses)
    compressed_responses = [f"{resp} ({count}x)" if count > 1 else resp for resp, count in response_counts.items()]
    final_function_calls.append({"index": idx, "available": compressed_responses})

# Save optimized function calls
output_path = "function-calls.json"
with open(output_path, "w") as f:
    json.dump(final_function_calls, f, indent=4)




Parsing date_time: '09-19-0000 13:30-14:45'
Fixing missing year for date '09-19-0000 13:30-14:45'
Generated Function Call: {'index': 0, 'available': 'no'}
Parsing date_time: '09-19-0000 15:00-16:15'
Fixing missing year for date '09-19-0000 15:00-16:15'
Generated Function Call: {'index': 0, 'available': 'no'}
Parsing date_time: '09-19-0000 16:30-17:45'
Fixing missing year for date '09-19-0000 16:30-17:45'
Generated Function Call: {'index': 0, 'available': 'no'}
Parsing date_time: '09-26-0000 13:30-14:45'
Fixing missing year for date '09-26-0000 13:30-14:45'
Generated Function Call: {'index': 0, 'available': 'no'}
Parsing date_time: '09-26-0000 15:00-16:15'
Fixing missing year for date '09-26-0000 15:00-16:15'
Generated Function Call: {'index': 0, 'available': 'no'}
Parsing date_time: '09-26-0000 16:30-17:45'
Fixing missing year for date '09-26-0000 16:30-17:45'
Generated Function Call: {'index': 0, 'available': 'no'}
Parsing date_time: '10-02-0000 13:30-14:45'
Fixing missing year for da